In [ ]:
data = {
    'xlsx'     : '/mnt/d/autocls_e3/classref/refexp32_vec.shp',
    'sheet'    : None,
    'pxid'     : 'pxid',
    'classcol' : 'rdv_class'
}

expname = 'exp33'

expdesc = 'Roads classes classification'

db = 'gthruth_valeflores_v3'

In [ ]:
import pandas as pd

from glass.rd import tbl_to_obj
from glass.prop.sql import check_last_id
from glass.wt.sql import df_to_db
from glass.prop.df import is_shp
from glass.rd.shp import shp_to_obj

In [ ]:
# Read data with association between pixels and classes
if is_shp(data['xlsx']):
    ddf = shp_to_obj(data['xlsx'])
else:
    ddf = tbl_to_obj(data['xlsx'], sheet=data['sheet'])

In [ ]:
mcols = [data['pxid'], data['classcol']]
ddf.drop([c for c in ddf.columns.values if c not in mcols], axis=1, inplace=True)

In [ ]:
ddf

In [ ]:
# Check last ID's
lid_exp = check_last_id(db, 'eid', 'experiments')
lid_cls = check_last_id(db, 'expclsid', 'expclasses')

In [ ]:
# Get experiment DF
edf = pd.DataFrame([[lid_exp+1, expname, expdesc]], columns=['eid', 'slug', 'description'])

# Get classes DF
expclasses = ddf[data['classcol']].unique()

df_classes = pd.DataFrame(expclasses, columns=['nameval'])

df_classes['codeval'] = df_classes.index + 1

df_classes['expclsid'] = lid_cls + 1 + df_classes.index
df_classes['expfk'] = lid_exp+1

In [ ]:
edf

In [ ]:
df_classes

In [ ]:
# Get association between pixels and experiment classes
ddf = ddf.merge(df_classes, how='left', left_on=data['classcol'], right_on='nameval')

In [ ]:
ddf

In [ ]:
ddf = ddf[[data['pxid'], 'expclsid']]

In [ ]:
ddf.rename(columns={
    data['pxid'] : 'pxid',
    'expclsid' : 'eclsid'
}, inplace=True)

In [ ]:
# Go for it
# Send everything to the database

df_to_db(db, edf, 'experiments', append=True)
df_to_db(db, df_classes, 'expclasses', append=True)
df_to_db(db, ddf, 'pxexpclasses', append=True)